In [1]:
# Cell 1: Install GDK
!pip install google-adk nest_asyncio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 5.8 MB/s eta 0:00:00a 0:00:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 202

In [17]:
# Cell 2: Setup API Key and model
import os
import asyncio
import nest_asyncio
nest_asyncio.apply()

from google.adk.agents import Agent, SequentialAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import google_search

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

os.environ["GOOGLE_API_KEY"] = user_secrets.get_secret("MEAL_PLANNER_KEY")

MODEL_ID = "gemini-2.0-flash"
APP_NAME = "meal_planner"
USER_ID = "user_123"
SESSION_ID = "session_001"

In [18]:
# Cell 4: Define Sub-Agents
dietary_analyzer = Agent(
    name="DietaryAnalyzer",
    model=MODEL_ID,
    description="Analyzes dietary preferences",
    instruction="""Extract user dietary info: allergies, diet type, cuisine preferences, meals needed.
    Output a structured summary.""",
)

recipe_finder = Agent(
    name="RecipeFinder",
    model=MODEL_ID,
    description="Finds recipes matching requirements",
    instruction="""Use Google Search to find 5 real recipes matching the dietary requirements.
    Return recipe names with ingredients list.""",
    tools=[google_search],
)

shopping_list_agent = Agent(
    name="ShoppingListGenerator",
    model=MODEL_ID,
    description="Creates shopping lists",
    instruction="""Based on the recipes provided, create a consolidated shopping list.
    Group items by category: produce, dairy, proteins, pantry.
    Combine duplicate ingredients and include quantities.""",
)

In [19]:
# Cell 5: Create Sequential Pipeline
meal_planner = SequentialAgent(
    name="WeeklyMealPlanner",
    sub_agents=[dietary_analyzer, recipe_finder, shopping_list_agent],
)

In [20]:
# Cell 6: Session & Runner
session_service = InMemorySessionService()

async def setup():
    session = await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={"preferences": {"diet": "vegetarian", "allergies": ["nuts"]}}
    )
    return session

session = asyncio.run(setup())

runner = Runner(
    agent=meal_planner,
    app_name=APP_NAME,
    session_service=session_service
)

In [21]:
# Cell 7: Run the Agent
async def call_agent(query: str):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    async for event in runner.run_async(
        user_id=USER_ID,
        session_id=SESSION_ID,
        new_message=content
    ):
        if event.is_final_response():
            if event.content and event.content.parts:
                print(event.content.parts[0].text)

In [22]:
# Cell 8: Test (Kaggle/Jupyter compatible)
request = """Plan my meals for this week. 
I'm vegetarian, allergic to nuts, and love Mediterranean food.
I need 5 dinner recipes."""

asyncio.run(call_agent(request))

```json
{
  "dietary_info": {
    "allergies": [
      "nuts"
    ],
    "diet_type": "vegetarian",
    "cuisine_preferences": [
      "Mediterranean"
    ],
    "meals_needed": [
      "5 dinners"
    ]
  }
}
```

Okay, I can help you find 5 vegetarian and nut-free Mediterranean dinner recipes for the week. Here are the recipes with their ingredients:


Okay, here is a consolidated shopping list based on the recipes provided, grouped by category:

**Produce:**

*   Cucumbers
*   Fresh vegetables (variety for soup and stuffed peppers, specify: zucchini, onions)
*   Bell peppers
*   Spinach
*   Garlic
*   Tomatoes
*   Herbs (variety for stuffed peppers, orzo, specify: oregano)
*   Lemon

**Dairy:**

*   Feta cheese

**Proteins:**

*   Chickpeas
*   Lentils
*   White beans
*   Cannellini beans

**Pantry:**

*   Rice
*   Orzo pasta
*   Sun-dried tomatoes
*   Cumin
*   Spices (for stuffed peppers)
*   Vegan pesto (check ingredients for nut-free)
*   Olives

